In [ ]:
import sys
sys.path.append('../../doclabels')
import plosdata
import json
import numpy as np

In [224]:
reload(plosdata)

<module 'plosdata' from 'plosdata.pyc'>

In [225]:
reload(plosapi)

<module 'plosapi' from 'plosapi.pyc'>

In [ ]:
plosdata.response_map

In [165]:
import plosapi
for docs in plosapi.sample("Biology and life sciences", 1000, 500):
    print('haha')

INFO:plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=Biology%20and%20life%20sciences&start=0&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM


0
haha

INFO:plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=Biology%20and%20life%20sciences&start=500&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM



500
haha


In [226]:
plosdata.save_source('./data', prefix='testsmall', limit=1000, increment=500)

INFO:plosdata:1000 documents to be saved for the subject: Biology and life sciences
INFO:prototype.plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=subject%3A%22Biology%20and%20life%20sciences%22&start=0&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM
INFO:prototype.plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=subject%3A%22Biology%20and%20life%20sciences%22&start=500&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM


KeyboardInterrupt: 

In [ ]:
text, response = plosdata.process_source('data/small-people_and_places.sample')

In [ ]:
filenames = [subject.replace(" ", "_").lower() for subject in plosdata.subject_areas]
file_list = ['{}/{}-{}.sample'.format('data', 'small', filename) for filename in filenames]
padded, responses = plosdata.process_sources(file_list)

In [ ]:
with open('small-padded', 'w') as f:
    json.dump(padded, f)

In [ ]:
with open('small-responses', 'w') as f2:
    json.dump(responses, f2)

In [ ]:
padded, responses= plosdata.load_processed('small-padded', 'small-responses')

In [ ]:
vocab = plosdata.build_vocabulary(padded)

In [ ]:
# Load Word2Vec
import gensim
model = gensim.models.Word2Vec.load_word2vec_format('../GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# Build word vectors space
word_vec_space = plosdata.build_word_vec_space(vocab, model)

In [ ]:
# load word vec space from file
word_vec_space = np.load('./data/small-word_vec_space.npy')

In [ ]:
# Build word vec index
word_vec_ind = plosdata.build_word_vec_index(vocab, word_vec_space)

In [ ]:
x, y = plosdata.build_input(vocab, padded, responses)

In [ ]:
x.shape

In [ ]:
np.savez('./data/small-input', x=x, y=y)

In [ ]:
np.savez('./data/small-word_vec_ind', word_vec_ind)

In [ ]:
# Modelling
len(word_vec_ind[0])

In [ ]:
embeddings = plosdata.create_embeddings(word_vec_ind)

In [ ]:
np.savez('./data/small-embeddings', embeddings)

In [ ]:
embeddings.shape[1]

In [ ]:
xandy = np.load('./data/small-embeddings.npz')

In [ ]:
xandy.files

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import plosdata
import logging

from doc_cnn import DocCNN


logger = logging.getLogger(__name__)
logging.basicConfig(filename='cnn_train.log', filemode='w', level=logging.DEBUG)



In [ ]:
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")


In [ ]:
len(word_vec_ind)

In [ ]:
raw_docs = map(lambda x: " ".join(x).replace(" <PAD/>", ""),padded)

In [ ]:
raw_docs[1]

In [ ]:
y_raw = map(lambda x: x.index(1) ,responses)

In [ ]:
list(shuffle_indices)

In [ ]:
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y_raw)))
x_raw_shuffled = np.asarray(raw_docs)[shuffle_indices]
y_raw_shuffled = np.asarray(y_raw)[shuffle_indices]
# Split train/test set
# TODO: This is very crude, should use cross-validation
x_raw_train, x_raw_test = x_raw_shuffled[:-1000], x_raw_shuffled[-1000:]
y_raw_train, y_raw_test = y_raw_shuffled[:-1000], y_raw_shuffled[-1000:]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
X_train_count = vectorizer.fit_transform(list(x_raw_train))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)
X_train_tfidf.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_raw_train)

In [ ]:
X_new_counts = vectorizer.transform(list(x_raw_test))
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(list(x_raw_test), predicted):
    print('%r => %s' % (doc, y_raw_test[category]))

In [ ]:
# Prediction accuracy
np.mean(predicted==y_raw_test)

In [ ]:
# Training accuracy
np.mean(x_raw_test)

In [ ]:
y_raw_test

In [213]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='squared_hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf.fit(list(x_raw_train), y_raw_train)
predicted_svm = text_clf.predict(x_raw_test)
np.mean(predicted_svm == y_raw_test)

0.68600000000000005

In [115]:
y_raw_train

array([6, 8, 9, ..., 9, 4, 4])

In [117]:
text_clf

Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        st...     penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))])

In [118]:
# Accuracy on training set
np.mean(text_clf.predict(x_raw_train) == y_raw_train)

0.86333333333333329

###### It means more data should be fed to tackle overfitting!

In [204]:
plosdata.save_source('./data', prefix='large', limit=10000)


INFO:plosdata:10000 documents to be saved for the subject: Biology and life sciences
INFO:prototype.plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=subject%3A%22Biology%20and%20life%20sciences%22&start=0&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM
INFO:prototype.plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=subject%3A%22Biology%20and%20life%20sciences%22&start=500&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM
INFO:prototype.plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=subject%3A%22Biology%20and%20life%20sciences%22&start=1000&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM
INFO:prototype.plosapi:making request to http://api.plos.org/search?fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&rows=500&q=subject%3A%22Biology%20and%20life%20sciences%22

In [209]:
filenames = [subject.replace(" ", "_").lower() for subject in plosdata.subject_areas]
file_list_l = ['{}/{}-{}.sample'.format('data', 'large', filename) for filename in filenames]
padded_l, responses_l, vocab_l = plosdata.process_sources(file_list_l)

processing: data/large-biology_and_life_sciences.sample


ValueError: Extra data: line 1 column 1247889 - line 1 column 24175300 (char 1247888 - 24175299)

In [210]:
tryload = plosdata.load_source('data/large-biology_and_life_sciences.sample')

ValueError: Extra data: line 1 column 1247889 - line 1 column 24175300 (char 1247888 - 24175299)

In [ ]:
# now use larger data set

raw_docs_l = map(lambda x: " ".join(x).replace(" <PAD/>", ""),padded_l)
y_raw_l = map(lambda x: x.index(1) ,responses_l)
np.random.seed(10)
shuffle_indices_l = np.random.permutation(np.arange(len(y_raw_l)))
x_raw_shuffled_l = np.asarray(raw_docs_l)[shuffle_indices_l]
y_raw_shuffled_l = np.asarray(y_raw_l)[shuffle_indices_l]
# Split train/test set
x_raw_train_l, x_raw_test_l = x_raw_shuffled_l[:-2000], x_raw_shuffled_l[-2000:]
y_raw_train_l, y_raw_test_l = y_raw_shuffled_l[:-2000], y_raw_shuffled_l[-2000:]


In [217]:
text_clf_l = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='squared_hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf_l.fit(list(x_raw_train_l), y_raw_train_l)
predicted_svm_l = text_clf_l.predict(x_raw_test_l)
np.mean(predicted_svm_l == y_raw_test_l)

0.71950000000000003

In [218]:
np.mean(text_clf_l.predict(x_raw_train_l) == y_raw_train_l)

0.96255119953188994

In [190]:
# get numFound
for subject in plosdata.subject_areas:
    print(plosapi.get_num({'q': 'subject:\"{}\"'.format(subject)}))

INFO:plosapi:making request to http://api.plos.org/search?q=subject%3A%22Biology%20and%20life%20sciences%22&fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM
INFO:plosapi:making request to http://api.plos.org/search?q=subject%3A%22Computer%20and%20information%20sciences%22&fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM


160018
13940

INFO:plosapi:making request to http://api.plos.org/search?q=subject%3A%22Earth%20sciences%22&fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM



12280

INFO:plosapi:making request to http://api.plos.org/search?q=subject%3A%22Ecology%20and%20environmental%20sciences%22&fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM


...


105846

INFO:plosapi:making request to http://api.plos.org/search?q=subject%3A%22Science%20policy%22&fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM



418

INFO:plosapi:making request to http://api.plos.org/search?q=subject%3A%22Social%20sciences%22&fq=doc_type%3Afull%20AND%20article_type%3A%22research%20article%22&wt=json&api_key=a1TjkHdh2NzVd6jhsDzM



20306


In [220]:
np.mean([1,2,3,4])

2.5

In [227]:
with open('{}-vocab'.format('large'), 'r') as f3:
    vocab_test = json.load(f3)

IOError: [Errno 2] No such file or directory: 'large-vocab'

In [229]:
with open('small-padded', 'w') as fp:
    json.dump(padded, fp)